# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [1]:
# import libraries
import pandas as pd
import numpy as np
from sqlalchemy import create_engine
import string
import re
import nltk

In [2]:
nltk.download(['punkt', 'wordnet', 'stopwords'])

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [3]:
# load data from database
engine = create_engine('sqlite:///DisasterResponse.db')
df = pd.read_sql_table('clean_messages', engine)
df

id                                            message  \
0          2  Weather update - a cold front from Cuba that c...   
1          7            Is the Hurricane over or is it not over   
2          8                    Looking for someone but no name   
3          9  UN reports Leogane 80-90 destroyed. Only Hospi...   
4         12  says: west side of Haiti, rest of the country ...   
5         14             Information about the National Palace-   
6         15                     Storm at sacred heart of jesus   
7         16  Please, we need tents and water. We are in Sil...   
8         17    I would like to receive the messages, thank you   
9         18  I am in Croix-des-Bouquets. We have health iss...   
10        20  There's nothing to eat and water, we starving ...   
11        21  I am in Petionville. I need more information r...   
12        22  I am in Thomassin number 32, in the area named...   
13        24  Let's do it together, need food in Delma 75, i...   
14        25  More information on the 4636 number in order f...   
15        26  A Comitee in Delmas 19, Rue ( street ) Janvier...   
16        27  We need food and water in Klecin 12. We are dy...   
17        28  are you going to call me or do you want me to ...   
18        30     I don't understand how to use this thing 4636.   
19        31  I would like to know if the earthquake is over...   
20        32  I would like to know if one of the radio ginen...   
21        33                     I'm in Laplaine, I am a victim   
22        34  There's a lack of water in Moleya, please info...   
23        35  Those people who live at Sibert need food they...   
24        36  I want to say hello, my message is to let you ...   
25        37                 Can you tell me about this service   
26        38  People I'm at Delma 2, we don't anything what ...   
27        39  We are at Gressier we needs assistance right a...   
28        41  How can we get water and food in Fontamara 43 ...   
29        42  We need help. Carrefour has been forgotten com...   
...      ...                                                ...   
26009  30080  Some 200,000 Chechens displaced by Russian mil...   
26010  30081  A general election held in Mongolia in July 20...   
26011  30082  Until water levels fully recede and infrastruc...   
26012  30083  Grawa, her place of birth, has been known for ...   
26013  30084  The families, who were previously living in ca...   
26014  30085  Some 5 or 6 mt of sorghum seed were taken when...   
26015  30086  He highlighted how the government is using the...   
26016  30087  Police have dropped charges against seven men ...   
26017  30088  Polio vaccination teams have repeatedly come u...   
26018  30089  The most commonly reported ailments at this ti...   
26019  30090  We are the military -- we don't do politics, a...   
26020  30091  (a) The second repatriation convoy of Somali r...   
26021  30093  Due to the storm, squalls with rough seas may ...   
26022  30094  WHO and the Ministry of Health will distribute...   
26023  30095  I held my son and tried to climb up to the roo...   
26024  30096  technical personnel for disease outbreak surve...   
26025  30097  Denied an effective voice and largely ignored ...   
26026  30098  In southern Pakistan's Baluchistan Province, U...   
26027  30100  (b) UNHCR expects to resume the facilitated vo...   
26028  30101  At one point, boxes had to be manhandled almos...   
26029  30102  Police said they expected the toll to rise fur...   
26030  30103  ADRA was granted general consultative status b...   
26031  30104  DEC member agencies are urgently seeking to in...   
26032  30105  Enhancing information exchange, delineating cl...   
26033  30106  NB: IFRC is reportedly stationing a heavy lift...   
26034  30107  "The pattern is always the same: steal, loot, ...   
26035  30108  Sirajganj, Bangladesh. When the rain-swollen J...   
26036  30109  Damage to the Nacala rail corridor by the r

In [4]:
# Define X and Y variables
X = df['message']
Y = df.drop(['id', 'message', 'original', 'genre'], axis=1)

### 2. Write a tokenization function to process your text data

In [5]:
stop_words = nltk.corpus.stopwords.words("english")
lemmatizer = nltk.stem.wordnet.WordNetLemmatizer()
remove_punc_table = str.maketrans('', '', string.punctuation)

In [6]:
def tokenize(text):
    # Remove punctuation and make lower case.
    text = text.translate(remove_punc_table).lower()
    
    tokens = nltk.word_tokenize(text)
    
    # Lemmatize and remove stop words.
    return [lemmatizer.lemmatize(word) for word in tokens if word not in stop_words]
    pass

### 3. Build a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. You may find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [7]:
# Import classes from ML labiraries
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.multioutput import MultiOutputClassifier
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.metrics import classification_report



In [8]:
forest_clf = RandomForestClassifier(n_estimators=10)

In [9]:
pipeline = Pipeline([
                    ('tfidf', TfidfVectorizer(tokenizer=tokenize)),
                    ('forest', MultiOutputClassifier(forest_clf))
                    ])

### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

In [10]:
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.33, random_state=42)
pipeline.fit(X_train, Y_train)

Pipeline(memory=None,
     steps=[('tfidf', TfidfVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), norm='l2', preprocessor=None, smooth_idf=True,
 ...oob_score=False, random_state=None, verbose=0,
            warm_start=False),
           n_jobs=1))])

### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

In [11]:
Y_pred = pipeline.predict(X_test)
for i, col in enumerate(Y_test):
    print(col)
    print(classification_report(Y_test[col], Y_pred[:, i]))

related
             precision    recall  f1-score   support

        0.0       0.31      0.15      0.20      2040
        1.0       0.77      0.89      0.83      6497
        2.0       0.17      0.02      0.03        56

avg / total       0.65      0.71      0.67      8593

request
             precision    recall  f1-score   support

        0.0       0.84      0.97      0.90      7117
        1.0       0.37      0.08      0.13      1476

avg / total       0.76      0.82      0.77      8593

offer
             precision    recall  f1-score   support

        0.0       0.99      1.00      1.00      8547
        1.0       0.00      0.00      0.00        46

avg / total       0.99      0.99      0.99      8593

aid_related
             precision    recall  f1-score   support

        0.0       0.60      0.79      0.69      5105
        1.0       0.44      0.24      0.31      3488

avg / total       0.54      0.57      0.53      8593

medical_help
             precision    recall  f1-sco

/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


### 6. Improve your model
Use grid search to find better parameters. 

In [12]:
# Grid search from sklearn

from sklearn.model_selection import GridSearchCV

In [ ]:
parameters = {
    #'tfidf__ngram_range': ((1, 1), (1, 2)),
    #'tfidf__max_df': (0.8, 1.0),
    #'tfidf__max_features': (None, 10000),
    #'forest__estimator__n_estimators': [10, 50],
    #'forest__estimator__min_samples_split': [2, 4]
}

cv = GridSearchCV(pipeline, parameters, cv=3, n_jobs=-1)
cv.fit(X_train, Y_train)

### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

In [ ]:
from sklearn.metrics import accuracy_score, precision_score, recall_score

In [ ]:
def evaluate_model(model, X_test, Y_test):
    Y_pred = model.predict(X_test)
    print(classification_report(Y_test, Y_pred, target_names=category_names))
    
# print('Accuracy, Precision and Recall Resulst)

print('Accuracy: ', accuracy_score(Y_test, Y_pred))
print('Precision: ', precision_score(Y_test, Y_pred, average='weighted'))
print('Recall: ', recall_score(Y_test, Y_pred, average='weighted'))

### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

In [ ]:
from sklearn.preprocessing import OneHotEncoder, FunctionTransformer
from sklearn.compose import ColumnTransformer
from sklearn.decomposition import TruncatedSVD

In [ ]:
def is_null(series):

    def is_null_base(s):
        res = 0 if s is None else 1
        return res
    
    final = pd.DataFrame(series.applymap(is_null_base))
    return final

In [ ]:
categorical_features = ['genre']
categorical_transform = OneHotEncoder()

original = ['original']
original_transform = FunctionTransformer(is_null, validate=False)

text = 'message'
text_transform = Pipeline([('tfidf', TfidfVectorizer(tokenizer=tokenize)),
                           ('best', TruncatedSVD(n_components=50))])


preprocessor = ColumnTransformer(transformers=[
                                ('cat', categorical_transform, categorical_features),
                                ('orig', original_transform, original),
                                ('tfidf', text_transform, 'message')
                                              ])


In [ ]:
preprocessor.fit_transform(X_2)

Note # This work will help to check how the code for the webserver would work. 

In [ ]:
clf = forest_clf
pipeline_2 = Pipeline([
                    ('tfidf', TfidfVectorizer(tokenizer=tokenize)),
                    ('best', TruncatedSVD(n_components=100)),
                    ('clf', MultiOutputClassifier(clf))
                      ])

# Test to check the performance

pipeline_2.fit(X_train, Y_train)

print(pipeline_2.steps)

# Model evaluation task

evaluate_model(pipeline_2, X_test, Y_test)

In [ ]:
# Use grid search to find better parameters and improve the model

param_grid = {
    'tfidf__ngram_range': ((1, 1), (1, 2)),
    'tfidf__max_df': (0.8, 1.0),
    'tfidf__max_features': (None, 10000),
    'best__n_components': (10, 50, 100),
    'clf__estimator__n_estimators': [50, 100],
    'clf__estimator__min_samples_split': [2, 4]
}

In [ ]:
cv_2 = GridSearchCV(pipeline_2, param_grid, cv=3, verbose=10, n_jobs=-1)

cv_2.fit(X_train, Y_train)

In [ ]:
from pprint import pprint

pprint(cv_2.best_estimator_.steps)


In [ ]:
evaluate_model(cv_2.best_estimator_, X_test, Y_test)

### 9. Export your model as a pickle file

In [ ]:
import joblib
joblib.dump(cv_2.best_estimator_, 'classifier.pkl')

### 10. Use this notebook to complete `train.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.